# Data import

data를 가져오는 부분

In [12]:
import csv
import numpy as np

with open('train.csv','r') as dest_f:
    data_iter = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data = [data for data in data_iter]
data_array = np.asarray(data, dtype = 'str')
print(data_array)
#print(data_array.shape)

[['PassengerId' 'Survived' 'Pclass' ... 'Fare' 'Cabin' 'Embarked']
 ['1' '0' '3' ... '7.25' '' 'S']
 ['2' '1' '1' ... '71.2833' 'C85' 'C']
 ...
 ['889' '0' '3' ... '23.45' '' 'S']
 ['890' '1' '1' ... '30' 'C148' 'C']
 ['891' '0' '3' ... '7.75' '' 'Q']]


data의 한글부분을 수치화한다.
(남자는 0, 여자는 1로)

In [13]:
quantifi_data = data_array[1:,:]

for i in range(quantifi_data.shape[0]):
    if quantifi_data[i][4]=='male':
        quantifi_data[i][4]=float(0)
    elif quantifi_data[i][4]=='female':
        quantifi_data[i][4]=float(1)

### 나이 데이터의 빈 공간을 채우는 방법을 찾아보자

방법 1. 1912년 미국의 기대수명을 이용한다. 남자의 경우 51.5살, 여자의 경우 55.9살이라는 데이터를 이용한다.
###### http://www.demog.berkeley.edu/~andrew/1918/figure2.html

In [14]:
for i in range(quantifi_data.shape[0]):
    if quantifi_data[i][5]=='':
        if quantifi_data[i][4]:
            quantifi_data[i][5]=float(51.5)
        else:
            quantifi_data[i][5]=float(55.9)
            
print(quantifi_data[5])

['6' '0' '3' 'Moran, Mr. James' '0.0' '51.5' '0' '0' '330877' '8.4583' ''
 'Q']


test 할 데이터 또한 받아온후 동일 작업을 시행한다

In [15]:
with open('test.csv','r') as dest_f:
    data_iter1 = csv.reader(dest_f, 
                           delimiter = ',', 
                           quotechar = '"')
    data1 = [data1 for data1 in data_iter1]
data_array1 = np.asarray(data1, dtype = 'str')
print(data_array1)
#print(data_array.shape)

quantifi_data1 = data_array1[1:,:]

for i in range(quantifi_data1.shape[0]):
    if quantifi_data1[i][3]=='male':
        quantifi_data1[i][3]=0
    elif quantifi_data1[i][3]=='female':
        quantifi_data1[i][3]=1
        
for i in range(quantifi_data1.shape[0]):
    if quantifi_data1[i][4]=='':
        if quantifi_data1[i][3]:
            quantifi_data1[i][4]=float(51.5)
        else:
            quantifi_data1[i][4]=float(55.9)

[['PassengerId' 'Pclass' 'Name' ... 'Fare' 'Cabin' 'Embarked']
 ['892' '3' 'Kelly, Mr. James' ... '7.8292' '' 'Q']
 ['893' '3' 'Wilkes, Mrs. James (Ellen Needs)' ... '7' '' 'S']
 ...
 ['1307' '3' 'Saether, Mr. Simon Sivertsen' ... '7.25' '' 'S']
 ['1308' '3' 'Ware, Mr. Frederick' ... '8.05' '' 'S']
 ['1309' '3' 'Peter, Master. Michael J' ... '22.3583' '' 'C']]


각 데이터마다 모두 범위가 차이나므로 격차가 발생, nomalization이 필요하다.

In [16]:
def MinMaxScaler(data):
    numerator = data.astype(np.float32) - np.min(data.astype(np.float32), 0)
    denominator = np.max(data.astype(np.float32), 0) - np.min(data.astype(np.float32), 0)
    
    return numerator / (denominator + 1e-7)

In [17]:
for i in range(quantifi_data[:,9].shape[0]):
    quantifi_data[i,9] = float(quantifi_data[i,9])
for i in range(quantifi_data1[:,8].shape[0]):
    if quantifi_data1[i,8]!='':
        quantifi_data1[i,8] = float(quantifi_data1[i,8])
    else:
        quantifi_data1[i,8] = 0

In [18]:
#quantifi_data[:,2] = MinMaxScaler(quantifi_data[:,2])
#quantifi_data[:,4] = MinMaxScaler(quantifi_data[:,4])
quantifi_data[:,5] = MinMaxScaler(quantifi_data[:,5])
quantifi_data[:,9] = MinMaxScaler(quantifi_data[:,9])
#quantifi_data[:,6] = MinMaxScaler(quantifi_data[:,6])
#quantifi_data1[:,1] = MinMaxScaler(quantifi_data1[:,1])
#quantifi_data1[:,3] = MinMaxScaler(quantifi_data1[:,3])
quantifi_data1[:,4] = MinMaxScaler(quantifi_data1[:,4])
quantifi_data1[:,8] = MinMaxScaler(quantifi_data1[:,8])
#quantifi_data1[:,5] = MinMaxScaler(quantifi_data1[:,5])

#### 성별과 동승자수, Pclass, Parch, 나이로 분류해보자
#### x data size는 [datanum, 5(Pclass, 성별, 나이, 동승자수, Parch)]
#### y data size는 [datanum, 1]이다

In [19]:
import tensorflow as tf

x_data = np.vstack([quantifi_data[:,2], quantifi_data[:,4], quantifi_data[:,5], quantifi_data[:,6], quantifi_data[:,7], quantifi_data[:,9]]).T
y_data = quantifi_data[:,1].reshape(quantifi_data.shape[0],1)
x_test_data = np.vstack([quantifi_data1[:,1], quantifi_data1[:,3], quantifi_data1[:,4], quantifi_data1[:,5], quantifi_data1[:,6], quantifi_data1[:,8]]).T
test_prediction = np.zeros([quantifi_data1.shape[0],1])

X = tf.placeholder(tf.float32, [None, 6])
Y = tf.placeholder(tf.float32, [None, 1])

W1 = tf.Variable(tf.random_normal([6,6]), name='weight1')
b1 = tf.Variable(tf.random_normal([6]), name='bias1')
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([6,3]), name='weight2')
b2 = tf.Variable(tf.random_normal([3]), name='bias2')
layer2 = tf.sigmoid(tf.matmul(layer1, W2) + b2)

W3 = tf.Variable(tf.random_normal([3,1]), name='weight3')
b3 = tf.Variable(tf.random_normal([1]), name='bias3')
hypothesis = tf.sigmoid(tf.matmul(layer2, W3) + b3)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
#train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y),dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(10001):
        sess.run(train, feed_dict={X: x_data,Y: y_data})
        
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data,Y: y_data}), sess.run([W1, W2, W3]))
    
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data,Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)
    
    #이제 실제 데이터를 집어넣어 예측해보자
    test_prediction = sess.run([hypothesis], feed_dict={X: x_test_data})
    print(test_prediction)
    

0 0.8179662 [array([[ 1.2957768 ,  0.7256303 , -1.3579793 , -0.9252475 ,  1.5730941 ,
         0.68312246],
       [ 0.04648663, -0.69492805, -0.22518101,  0.41868106,  0.40109688,
         0.7854036 ],
       [ 0.20584267,  0.9945336 ,  0.37062258,  0.07291003, -0.50685406,
         0.50528985],
       [-0.18892351,  0.39008158,  0.5900302 , -0.03482021,  0.33972195,
        -0.33163813],
       [-1.0930775 , -0.11285238, -0.60413104, -0.49859223,  0.1662813 ,
         1.0618695 ],
       [ 0.09900609,  1.1322831 , -0.16972613, -0.47803912,  1.295576  ,
        -0.20217046]], dtype=float32), array([[ 0.1491521 ,  0.8931583 ,  0.10274908],
       [ 0.26066136,  0.40755436, -1.6229959 ],
       [ 0.72572   , -0.45964804,  1.0577093 ],
       [ 0.27482638,  0.9785653 , -0.9877118 ],
       [ 3.279048  , -1.5360155 , -0.03939901],
       [ 0.11689436, -0.52813417,  0.07295927]], dtype=float32), array([[-1.178391 ],
       [ 1.8940313],
       [ 1.2234918]], dtype=float32)]
100 0.6151516 [

800 0.39760613 [array([[ 1.91849756e+00,  8.87085736e-01,  1.79671776e+00,
         1.76648033e+00,  1.96442997e+00,  2.91052639e-01],
       [ 1.85821757e-01, -6.95592821e-01, -5.52240944e+00,
         2.27237828e-02,  3.03976774e-01, -1.11242700e+00],
       [ 5.78182578e-01,  1.01662910e+00, -2.02629585e-02,
         2.76811093e-01,  1.33757545e-02,  4.79262412e-01],
       [ 5.19224703e-01,  3.90558600e-01,  9.61518064e-02,
         5.16302109e+00,  5.33717036e-01,  1.84662247e+00],
       [-6.46059275e-01, -5.72193600e-02,  7.66261160e-01,
        -4.73859310e+00,  4.14876550e-01, -1.54938642e-03],
       [ 4.82232749e-01,  1.55210662e+00,  7.30194151e-03,
        -4.72872049e-01,  1.57774091e+00, -2.71971188e-02]], dtype=float32), array([[-0.68757445,  1.9528519 , -0.21784796],
       [-0.57719356,  1.4686863 , -1.9436784 ],
       [ 2.9672306 , -2.904152  ,  0.50988626],
       [ 1.0357089 , -0.82265395, -1.5928622 ],
       [ 0.87131244,  0.90567106, -0.43759033],
       [ 1.47

1700 0.38644186 [array([[ 1.9178343e+00,  8.8711458e-01,  2.7272859e+00,  2.1479087e+00,
         2.3911076e+00,  5.7634366e-01],
       [ 9.2612565e-02, -6.9559282e-01, -6.7808046e+00,  2.9482543e+00,
         3.0856186e-01,  2.1736851e+00],
       [ 5.4359698e-01,  1.0166330e+00, -6.3193846e-03,  3.6508039e-01,
        -1.6753335e-01,  8.3971232e-01],
       [ 5.6170952e-01,  3.9055860e-01,  1.3176662e-01,  5.6817083e+00,
         5.3340244e-01,  2.6416037e+00],
       [-6.4451247e-01, -5.7209395e-02,  7.7186465e-01, -6.6375632e+00,
         4.1294017e-01, -7.4346048e-01],
       [ 4.8248816e-01,  1.5521826e+00,  1.1902311e-02, -4.9618888e-01,
         1.5535462e+00, -4.4839464e-02]], dtype=float32), array([[-1.025891  ,  2.058969  , -0.3002499 ],
       [-0.91572577,  1.5748785 , -2.026148  ],
       [ 3.5408852 , -3.444547  ,  0.46156508],
       [ 3.2295032 ,  0.8839237 , -1.627093  ],
       [-0.3255555 ,  2.1278849 , -0.6724849 ],
       [ 2.428589  , -3.1506488 , -0.5011682 ]],

2500 0.38180134 [array([[ 1.9091280e+00,  8.8711458e-01,  3.2498960e+00,  2.8191786e+00,
         3.2560017e+00,  6.9156075e-01],
       [ 2.1855615e-02, -6.9559282e-01, -8.3347597e+00,  5.4319673e+00,
         9.1714859e-01,  3.2430742e+00],
       [ 5.0784481e-01,  1.0166330e+00, -9.7339889e-03,  5.6607896e-01,
        -3.1188908e-01,  1.0806835e+00],
       [ 5.8604932e-01,  3.9055860e-01,  9.4968192e-02,  8.6253128e+00,
         3.6495894e-01,  3.4276199e+00],
       [-6.4797157e-01, -5.7209395e-02,  1.0252694e+00, -9.7496061e+00,
        -4.7664034e-01, -1.1647255e+00],
       [ 4.7933391e-01,  1.5521826e+00,  1.4577670e-02, -7.6253265e-01,
         1.3386765e+00, -5.6319080e-02]], dtype=float32), array([[-1.5471646 ,  2.0928228 , -0.3563842 ],
       [-1.4372693 ,  1.6087482 , -2.082328  ],
       [ 4.6522098 , -3.719524  ,  0.4483597 ],
       [ 4.7002754 ,  2.481288  , -1.6975106 ],
       [-1.0946212 ,  2.8672972 , -0.84701777],
       [ 2.6404624 , -3.976885  , -0.48960215]],

3300 0.3777043 [array([[ 1.9126755e+00,  8.8711458e-01,  3.6543267e+00,  3.3571303e+00,
         3.8252802e+00,  8.9769673e-01],
       [-2.8362142e-02, -6.9559282e-01, -9.1710835e+00,  7.5119538e+00,
         9.9458569e-01,  4.1108441e+00],
       [ 4.9249208e-01,  1.0166330e+00, -1.1349979e-02,  7.9166603e-01,
        -3.9508337e-01,  1.2986557e+00],
       [ 6.0896432e-01,  3.9055860e-01,  5.3583417e-02,  1.2254777e+01,
         9.0809122e-02,  4.0425515e+00],
       [-6.4567268e-01, -5.7209395e-02,  9.7707677e-01, -1.2599882e+01,
        -5.7150702e+00, -1.5153823e+00],
       [ 4.8082733e-01,  1.5521826e+00,  1.4780493e-02, -1.0782684e+00,
         1.6743714e+00, -6.7537770e-02]], dtype=float32), array([[-2.0737488 ,  2.083755  , -0.41175   ],
       [-1.9640505 ,  1.599676  , -2.137744  ],
       [ 5.73797   , -4.5133095 ,  0.44350243],
       [ 7.131495  ,  4.587121  , -1.6948421 ],
       [-1.7750897 ,  3.935609  , -1.0221676 ],
       [ 2.3590384 , -5.2195296 , -0.4686233 ]], 

...

6500 0.37282047 [array([[ 2.0108509e+00,  8.8711458e-01,  4.4706478e+00,  4.6466670e+00,
         6.2501621e+00,  1.9708594e+00],
       [-4.6223450e-02, -6.9559282e-01, -1.0335557e+01,  1.2278287e+01,
         3.2939661e+00,  5.8640885e+00],
       [ 5.6166810e-01,  1.0166330e+00, -1.1096886e-02,  1.3213441e+00,
        -8.4778351e-01,  2.0060847e+00],
       [ 6.6471219e-01,  3.9055860e-01,  2.4210436e-02,  2.1086029e+01,
        -6.1590880e-01,  5.9140811e+00],
       [-5.9414363e-01, -5.7209395e-02,  7.1900594e-01, -2.0628494e+01,
        -9.1993437e+00, -2.2470794e+00],
       [ 5.1901132e-01,  1.5521826e+00,  1.8126430e-02, -1.8575853e+00,
         3.5279536e+00, -1.0324500e-01]], dtype=float32), array([[-2.6313186 ,  1.9446512 , -0.5194668 ],
       [-2.5217247 ,  1.4605522 , -2.245553  ],
       [ 8.50772   , -8.056691  ,  0.57767695],
       [ 9.492987  ,  9.787065  , -1.8156948 ],
       [-2.63192   ,  9.169135  , -1.316996  ],
       [ 2.3500051 , -9.984534  , -0.14018472]],

7300 0.3723212 [array([[ 2.0458841e+00,  8.8711458e-01,  4.6635013e+00,  4.9365511e+00,
         6.9794521e+00,  1.8020474e+00],
       [-4.5828663e-02, -6.9559282e-01, -1.0621514e+01,  1.2780246e+01,
         3.5682540e+00,  6.4143600e+00],
       [ 5.7231581e-01,  1.0166330e+00, -1.1106225e-02,  1.3971583e+00,
        -9.4969887e-01,  2.0595698e+00],
       [ 6.7685938e-01,  3.9055860e-01,  2.3348164e-02,  2.2362139e+01,
        -6.8900150e-01,  6.2933364e+00],
       [-5.7452947e-01, -5.7209395e-02,  6.7770457e-01, -2.1857819e+01,
        -9.9318695e+00, -2.0516672e+00],
       [ 5.3281021e-01,  1.5521826e+00,  1.8940656e-02, -1.9634027e+00,
         3.9525223e+00, -1.0521685e-01]], dtype=float32), array([[ -2.66722   ,   1.8765464 ,  -0.5184929 ],
       [ -2.557627  ,   1.3924432 ,  -2.244579  ],
       [  8.802148  ,  -9.194972  ,   0.6174809 ],
       [  9.593424  ,  10.846277  ,  -1.8328744 ],
       [ -2.73193   ,  10.54167   ,  -1.2729659 ],
       [  2.3887987 , -10.888683  

8100 0.37077504 [array([[ 2.0878899e+00,  8.8711458e-01,  4.8503489e+00,  5.1851358e+00,
         8.1002588e+00,  1.9354415e+00],
       [-4.4148836e-02, -6.9559282e-01, -1.0985027e+01,  1.3369138e+01,
         3.6973503e+00,  6.6987658e+00],
       [ 5.8633614e-01,  1.0166330e+00, -1.2192266e-02,  1.4669935e+00,
        -1.0490938e+00,  2.0849364e+00],
       [ 6.8906361e-01,  3.9055860e-01,  2.4609506e-02,  2.3508488e+01,
        -7.3187137e-01,  6.2232094e+00],
       [-5.5137825e-01, -5.7209395e-02,  6.5106672e-01, -2.2984514e+01,
        -1.0588844e+01, -2.2363229e+00],
       [ 5.4937351e-01,  1.5521826e+00,  1.9652434e-02, -2.0583684e+00,
         4.4031267e+00, -1.0668414e-01]], dtype=float32), array([[ -2.6897805,   1.8279456,   0.0505744],
       [ -2.5801888,   1.3438405,  -1.6751343],
       [  8.979257 ,  -9.892013 ,   1.2894924],
       [  9.581538 ,  12.112713 ,  -0.6215   ],
       [ -2.8174162,  11.675059 ,   0.7125495],
       [  2.527775 , -12.058209 ,   2.6364975]],

8900 0.36792657 [array([[ 2.15724468e+00,  8.87114584e-01,  5.09093380e+00,
         5.38741159e+00,  1.05865555e+01,  1.53353727e+00],
       [ 1.95221640e-02, -6.95592821e-01, -1.11800737e+01,
         1.43749647e+01,  3.74766779e+00,  8.50100327e+00],
       [ 6.45916998e-01,  1.01663303e+00, -1.00648226e-02,
         1.57308376e+00, -1.18524837e+00,  2.09065557e+00],
       [ 6.89707577e-01,  3.90558600e-01,  3.03290375e-02,
         2.52700748e+01, -7.50758231e-01,  6.10637236e+00],
       [-5.14973462e-01, -5.72093949e-02,  5.25626004e-01,
        -2.45989323e+01, -1.10667372e+01, -1.89320314e+00],
       [ 5.78046262e-01,  1.55218256e+00,  1.99965276e-02,
        -2.21580482e+00,  5.13184309e+00, -1.05593264e-01]], dtype=float32), array([[ -2.7962532 ,   1.9056072 ,  -0.46540564],
       [ -2.6866615 ,   1.4215062 ,  -2.1911504 ],
       [  9.306448  ,  -9.704951  ,   5.046154  ],
       [  9.480392  ,  12.466811  ,  -2.178684  ],
       [ -2.9892845 ,  12.424786  ,   0.74348015

9700 0.36685556 [array([[ 2.2265487e+00,  8.8711458e-01,  5.2304173e+00,  5.6551471e+00,
         1.2617917e+01,  4.9804670e-01],
       [ 1.0267311e-01, -6.9559282e-01, -1.1465181e+01,  1.5343220e+01,
         3.6300499e+00,  1.0435002e+01],
       [ 7.2153705e-01,  1.0166330e+00, -1.0229892e-02,  1.6766241e+00,
        -1.4231863e+00,  2.0373967e+00],
       [ 6.7970181e-01,  3.9055860e-01,  3.5490777e-02,  2.6958176e+01,
        -7.5413537e-01,  6.1448994e+00],
       [-4.7984189e-01, -5.7209395e-02,  5.4028398e-01, -2.6184126e+01,
        -1.1547930e+01, -1.5826104e+00],
       [ 6.0761946e-01,  1.5521826e+00,  2.0444306e-02, -2.3659713e+00,
         6.1710515e+00, -1.0156357e-01]], dtype=float32), array([[ -2.8320916 ,   1.8773552 ,  -0.45202094],
       [ -2.7224996 ,   1.3932538 ,  -2.1777735 ],
       [  9.44635   ,  -8.9895935 ,   4.891224  ],
       [  9.991941  ,  12.490167  ,  -2.8609037 ],
       [ -3.0300808 ,  12.770925  ,   0.8640109 ],
       [  2.424973  , -14.154034 

test_prediction의 값을 0과 1로 정규화 시켜준다

In [9]:
for i in range(len(test_prediction[0])):
    if test_prediction[0][i] > 0.5:
        test_prediction[0][i] = 1
    else:
        test_prediction[0][i] = 0

print(test_prediction)

[array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.]

예측값들을 csv파일로 저장한다

In [10]:
f = open('prediction4.csv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f)
wr.writerow(["PassengerId", "Survived"])
for i in range(len(test_prediction[0])):
    wr.writerow([892+i, int(test_prediction[0][i])])
    
f.close()